# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

__План и работы и интерактивное содержание проекта:__

1. [Знакомство с данными](#start)
2. [Предобработка данных](#preprocessing)
 - [Обработка пропусков](#passes)
 - [Замена типа данных](#types)
 - [Обработка дубликатов](#duplicates)
 - [Лемматизация](#lemmatize)
 - [Категоризация](#categories)
3. [Ответы на вопросы кейса](#main)
4. [Общий вывод](#final)

<a id="start"></a>
## Знакомство с данными

In [1]:
#импортируем библиотеки
import pandas as pd
from pymystem3 import Mystem
m = Mystem() 

In [2]:
#открываем базу данных и ознакамливаемся с результатом
data = pd.read_csv('/datasets/data.csv')
display(data.sample(5))
data.info()
data.describe().T

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6233,0,340159.552353,66,Среднее,1,женат / замужем,0,F,пенсионер,0,160309.864002,покупка жилой недвижимости
10585,0,NaN,65,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,покупка недвижимости
8036,0,-197.927461,51,среднее,1,женат / замужем,0,M,сотрудник,0,218574.572655,автомобиль
7896,0,-1166.937340,49,среднее,1,женат / замужем,0,F,сотрудник,0,119933.334961,операции со своей недвижимостью
14167,2,NaN,51,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции с коммерческой недвижимостью


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


**Вывод**

Для ответа на вопросы кейса не требуется то количество информации, которое представлено в таблице. Столбцы `days_employed`, `dob_years`, `education`, `education_id`, `gender`. `familty_status` не потребуются, от них следует избавиться. 

Из интересующих нас столбцов наблюдаются явные пропуски в `total_income`.

<a id="preprocessing"></a>
## Предобработка данных

Перед началом работы удалим ненужные столбцы

In [3]:
data = data.drop(columns = ['days_employed', 'dob_years', 'education', 'education_id', 'gender', 'family_status_id'])

<a id="passes"></a>
### Обработка пропусков

Исследуем столбец `days_employed` на пропуски

In [4]:
data[data['total_income'].isna()].head(10)

,children,family_status,income_type,debt,total_income,purpose
12,0,гражданский брак,пенсионер,0,NaN,сыграть свадьбу
26,0,женат / замужем,госслужащий,0,NaN,образование
29,0,Не женат / не замужем,пенсионер,0,NaN,строительство жилой недвижимости
41,0,женат / замужем,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,гражданский брак,пенсионер,1,NaN,сыграть свадьбу
65,0,Не женат / не замужем,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,женат / замужем,пенсионер,0,NaN,покупка жилья для семьи
72,1,женат / замужем,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,женат / замужем,сотрудник,0,NaN,жилье
83,0,женат / замужем,сотрудник,0,NaN,жилье


Значение NaN в столбце `total_income` не является эквивалентом "нуля" так как в столбце `income_type` есть сотрудники и госслужащие, которые не могут не иметь дохода. Возможно это ошибка при заполнении данных. Требуется заполнить пустые значения медианой или средним значением. Рассмотрим оба варианта и выберем наиболее подходящий.

In [5]:
#смотрим медиану и среднее по столбцу total_income
data.groupby('income_type').agg({'total_income': ['mean','median']})

total_income               
                          mean         median
income_type                                  
безработный      131339.751676  131339.751676
в декрете         53829.130729   53829.130729
госслужащий      170898.309923  150447.935283
компаньон        202417.461462  172357.950966
пенсионер        137127.465690  118514.486412
предприниматель  499163.144947  499163.144947
сотрудник        161380.260488  142594.396847
студент           98201.625314   98201.625314

Медиана либо меньше, либо равна средней. Второе скорее всего происходит в ситуациях, когда значений рассматриваемого типа в `income_type` не более двух. Проверим какие именно типы имеют пропущенные значения и посчитаем их.

In [6]:
#считаем количество пропусков по столбцу income_type
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Категория "предприниматель" состоит всего из одной строки. Судя по показателю медианы и среднего, предпринимателей всего два, логичнее удалить эту строку, чем заполнять ее. Остальных заполним медианами.

Также добавим в данные столбец `autofill_id`, где значение 1 будет означать, что строка подверглась изменениям. Это необходимо для дальнейшей обработки дубликатов. Такие строки метод `.duplicated` может расценивать как дубликаты, хотя они таковыми не являются.

In [7]:
#создаем список типов дохода в которых есть пропуски
isna_income_types = data[data['total_income'].isna()]['income_type'].unique()

#заполняем все пропуски в доходе, кроме типа "предприниматель" медианами
for income_type in isna_income_types:
    if income_type != 'предприниматель':
        median = data[data['income_type'] == income_type]['total_income'].median()
        data.loc[(data['income_type'] == income_type) & (data['total_income'].isna()), 'autofill_id'] = 1        
        data.loc[(data['income_type'] == income_type) & (data['total_income'].isna()), 'total_income'] = median

#строку со значением "предприниматель" удаляем
data = data.dropna(subset = ['total_income']).reset_index(drop=True)

#проверка
print('---Проверка появления столбца autofill_id---')
display(data[data['autofill_id'] == 1].head(5))
print()
print('---Проверка заполнения пустых значений в столбце total_income---')
print()
data.info()

---Проверка появления столбца autofill_id---


,children,family_status,income_type,debt,total_income,purpose,autofill_id
12,0,гражданский брак,пенсионер,0,118514.486412,сыграть свадьбу,1.0
26,0,женат / замужем,госслужащий,0,150447.935283,образование,1.0
29,0,Не женат / не замужем,пенсионер,0,118514.486412,строительство жилой недвижимости,1.0
41,0,женат / замужем,госслужащий,0,150447.935283,сделка с подержанным автомобилем,1.0
55,0,гражданский брак,пенсионер,1,118514.486412,сыграть свадьбу,1.0



---Проверка заполнения пустых значений в столбце total_income---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21524 non-null  int64  
 1   family_status  21524 non-null  object 
 2   income_type    21524 non-null  object 
 3   debt           21524 non-null  int64  
 4   total_income   21524 non-null  float64
 5   purpose        21524 non-null  object 
 6   autofill_id    2173 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.1+ MB


Теперь изучим остальные столбцы на предмет неявных пропусков

In [8]:
print('--Столбец children--')
print(data['children'].value_counts())
print()
print('--Столбец family_status--')
print(data['family_status'].value_counts())
print()
print('--Столбец income_type--')
print(data['income_type'].value_counts())
print()
print('--Столбец debt--')
print(data['debt'].value_counts())
print()
print('--Столбец total_income значения меньше нуля--')
print(data[data['total_income'] < 0]['total_income'].count())
print()
print('--Стобец purpose--')
print(data['purpose'].value_counts())

--Столбец children--
 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

--Столбец family_status--
женат / замужем          12379
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

--Столбец income_type--
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

--Столбец debt--
0    19783
1     1741
Name: debt, dtype: int64

--Столбец total_income значения меньше нуля--
0

--Стобец purpose--
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции 

Столбец `children` имеет значение -1, что невозможно, так как он указывает количество детей. Рассмотрим какие именно категории `family_status` имеют значения столбца `children` -1

In [9]:
#значения столбца children -1 по столбцу family_status
data[data['children'] < 0]['family_status'].value_counts()

женат / замужем          29
Не женат / не замужем     5
гражданский брак          5
в разводе                 4
вдовец / вдова            4
Name: family_status, dtype: int64

Больше всего значений среди людей состоящих в браке и примерно поровну в остальных категориях. Такое поведение данных говорит скорее о том, что вместо -1 должно стоять значение 1, вопрос только с людьми со статусом "не женат / не замужем". Проверим как распределяются данные о наличии детей в диапозоне без ошибок.

In [10]:
#значение столбца children больше 0 по столбцу family_status
data[data['children'] > 0]['family_status'].value_counts()

женат / замужем          4851
гражданский брак         1420
Не женат / не замужем     543
в разводе                 407
вдовец / вдова            108
Name: family_status, dtype: int64

В диапозоне без ошибок люди со статусом "не женат / не замужем" также имеют детей. Распределение данных в рассматриваемом диапозоне похоже на распределение в диапозоне с ошибками, за некоторыми незначительными исключениями. Скорее всего значение -1 необходимо заменить на значение 1.

In [11]:
#заменяем значение в столбце children с -1 на 1
data['children'] = data['children'].replace(-1, 1)

#проверка
data[data['children'] < 0]['family_status'].value_counts()

Series([], Name: family_status, dtype: int64)

**Вывод**

- Столбец `total_income` имел явные пропуски. Они были заполнены медианными значениями с учетом категорий дохода
- Удалена одна строка с пустым значением в столбце `total_income`, где `income_type` соответствовал значению "предприниматель", это менее 1% от общего массива данных, на результат повлиять не должно 
- Выявлены ошибки в значениях столбца `children`, где присутствует значение -1. Поведение данных в диапозоне с ошибками примерно похоже на поведение данных в диапозоне без ошибок. Принято решение заменить заначение -1 на 1.

<a id="types"></a>
### Замена типа данных

Узнаем сколько памяти потребляет нынешняя база данных и сравним это значение по результату изменений типов данных

In [12]:
data.info(verbose = False, memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Columns: 7 entries, children to autofill_id
dtypes: float64(2), int64(2), object(3)
memory usage: 8.0 MB


Данные в столбце `total_income` имеет тип данных `float`. Для нашего исследования в этом нет никакой необходимости, а множество знаков после запятой мешают смотреть на таблицу и являются визуальным мусором. Избавимся от него, поменяв тип данных с `float` на `int`. 

In [13]:
#меняем float на int для данных из столбца total_income
data['total_income'] = data['total_income'].astype('uint32')

#проверка
data.head(5)

,children,family_status,income_type,debt,total_income,purpose,autofill_id
0,1,женат / замужем,сотрудник,0,253875,покупка жилья,NaN
1,1,женат / замужем,сотрудник,0,112080,приобретение автомобиля,NaN
2,0,женат / замужем,сотрудник,0,145885,покупка жилья,NaN
3,3,женат / замужем,сотрудник,0,267628,дополнительное образование,NaN
4,0,гражданский брак,пенсионер,0,158616,сыграть свадьбу,NaN


Для целей оптимизации есть смысл также совершить следующие действия: 
- Столбцы `children`, `debt` перевести в тип `uint8`
- Столбцы `family_status`, `income_type` в тип `category`

Сделаем это

In [14]:
data['children'] = data['children'].astype('uint8')
data['debt'] = data['debt'].astype('uint8')
data['family_status'] = data['family_status'].astype('category')
data['income_type'] = data['income_type'].astype('category')

Проверим потребление данных после оптимизации

In [15]:
data.info(verbose = False, memory_usage = 'deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Columns: 7 entries, children to autofill_id
dtypes: category(2), float64(1), object(1), uint32(1), uint8(2)
memory usage: 2.9 MB


**Вывод**

- Убрали лишние знаки после запятой в столбце `total_income`, что бы данные было проще воспринимать визуально
- Сменили тип данных в столбце `total_income` с `int64` на `uint32`
- Сменили тип данных в столбцах `children` и `debt` с `int64` на `uint8`
- Сменили тип данных в столбцах `family_status` и `income_type` с `object` на `category`
- Общее использование памяти снижено на 64% с 8 MB до 2.9 MB

<a id="duplicates"></a>
### Обработка дубликатов

Проверим данные на дубликаты. Для этого воспользуемся логической индексацией, где первым условием будет работа метода `.duplicated`, а вторым условием будет отсутствие значения в столбце `autofill_id`. Проще говоря, проверим, есть ли дубликаты в тех строках, где мы не меняли значение столбца `total_income` на этапе работы с пропусками 

In [16]:
#индекс дублированных строк в диапозоне данных, который мы не меняли в процессе работы с пропусками
duplicated_columns = data[(data.duplicated()) & (data['autofill_id'] != 1)].index.tolist()
duplicated_columns

[11532, 13258, 19477]

Обнаружено три дубликата. Избавимся от них

In [17]:
#убираем дубликаты 
data = data.drop(duplicated_columns).reset_index(drop = True)

#проверка
data[(data.duplicated()) & (data['autofill_id'] != 1)]

,children,family_status,income_type,debt,total_income,purpose,autofill_id


**Вывод**

- Проверили данные на дубликаты в том диапозоне, который мы не меняли в процессе работы с пропусками используя логическую индексацию
- Обнаружили три дубликата и выяснили их индексы
- Избавились от них, используя метод `.drop`

<a id="lemmatize"></a>
### Лемматизация

На стадии работы с неявными пропусками было обнаружено множество одинаковых по сути значений в столбце purpose. Применим метод лемматизации данных для создание более удобных для анализа категорий. В рассматриваемом дипозоне предлагается использовать следующие категории целей кредита: 
- Жилая недвижимость для всех предложений, где есть слово жилье (кроме ремонта)
- Прочая недвижимость для всех предложений, где есть слово недвижмость (кроме жилья и ремонта)
- Ремонт
- Свадьба
- Образование
- Автомобиль

In [18]:
#создаем функцию, которая превратит значения в столбце purpose в категории
def purpose_to_category(row):
    purpose = row['purpose']
    lemma = m.lemmatize(purpose)
    
    if 'ремонт' in lemma:
        return 'ремонт'
    if 'жилье' in lemma:
        return 'жилая недвижимость'
    if 'недвижимость' in lemma:
        return 'прочая недвижимость'
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    
    #используем для проверки
    return 'проверка'

In [19]:
# применяем функцию для создание столбца purpose_category
data['purpose_category'] = data.apply(purpose_to_category, axis = 1)
data.head(10)

,children,family_status,income_type,debt,total_income,purpose,autofill_id,purpose_category
0,1,женат / замужем,сотрудник,0,253875,покупка жилья,NaN,жилая недвижимость
1,1,женат / замужем,сотрудник,0,112080,приобретение автомобиля,NaN,автомобиль
2,0,женат / замужем,сотрудник,0,145885,покупка жилья,NaN,жилая недвижимость
3,3,женат / замужем,сотрудник,0,267628,дополнительное образование,NaN,образование
4,0,гражданский брак,пенсионер,0,158616,сыграть свадьбу,NaN,свадьба
5,0,гражданский брак,компаньон,0,255763,покупка жилья,NaN,жилая недвижимость
6,0,женат / замужем,компаньон,0,240525,операции с жильем,NaN,жилая недвижимость
7,0,женат / замужем,сотрудник,0,135823,образование,NaN,образование
8,2,гражданский брак,сотрудник,0,95856,на проведение свадьбы,NaN,свадьба
9,0,женат / замужем,сотрудник,0,144425,покупка жилья для семьи,NaN,жилая недвижимость


In [20]:
#проверяем, что среди уникальных значений столбца purpose_category нет значения "проверка"
data['purpose_category'].unique()

array(['жилая недвижимость', 'автомобиль', 'образование', 'свадьба',
       'прочая недвижимость', 'ремонт'], dtype=object)

**Вывод**

- Столбец `purpose` необходимо категоризировать с помощью лемматизации для упрощения анализа
- Для выполнения категоризации использована функция и создан новый столбец `purpose_category`

Были выбраны следующие категории: 
- Жилая недвижимость для всех предложений, где есть слово жилье (кроме ремонта)
- Прочая недвижимость для всех предложений, где есть слово недвижмость (кроме жилья и ремонта)
- Ремонт
- Свадьба
- Образование
- Автомобиль

<a id="categories"></a>
### Категоризация данных

Для простоты анализа данные необходимо категоризировать. В кейсе задан следующий вопрос: "есть ли зависимость между наличием детей и возвратом кредита в срок?". То есть нас интересует наличие или отсутствие детей, но не их количество. Необходимо ввести две категории по столбцу `children`:
- Есть дети
- Нет детей

In [21]:
#создаем функцию для категоризации столбца children
def children_to_category(row):
    children = row['children']
    
    if children > 0:
        return 'есть дети'
    return 'нет детей'

In [22]:
#применяем функцию для создания солбца children_category
data['children_category'] = data.apply(children_to_category, axis = 1)

#проверка
data.head(10)

,children,family_status,income_type,debt,total_income,purpose,autofill_id,purpose_category,children_category
0,1,женат / замужем,сотрудник,0,253875,покупка жилья,NaN,жилая недвижимость,есть дети
1,1,женат / замужем,сотрудник,0,112080,приобретение автомобиля,NaN,автомобиль,есть дети
2,0,женат / замужем,сотрудник,0,145885,покупка жилья,NaN,жилая недвижимость,нет детей
3,3,женат / замужем,сотрудник,0,267628,дополнительное образование,NaN,образование,есть дети
4,0,гражданский брак,пенсионер,0,158616,сыграть свадьбу,NaN,свадьба,нет детей
5,0,гражданский брак,компаньон,0,255763,покупка жилья,NaN,жилая недвижимость,нет детей
6,0,женат / замужем,компаньон,0,240525,операции с жильем,NaN,жилая недвижимость,нет детей
7,0,женат / замужем,сотрудник,0,135823,образование,NaN,образование,нет детей
8,2,гражданский брак,сотрудник,0,95856,на проведение свадьбы,NaN,свадьба,есть дети
9,0,женат / замужем,сотрудник,0,144425,покупка жилья для семьи,NaN,жилая недвижимость,нет детей


In [23]:
#проверка на предмет того, какие значения столбца children присутствуют в диапозоне данных, где значение столбца
#children_category равно "есть дети"
data[data['children_category'] == 'есть дети']['children'].unique()

array([ 1,  3,  2,  4, 20,  5], dtype=uint8)

In [24]:
#проверка на предмет того, какие значения столбца children присутствуют в диапозоне данных, где значение столбца
#children_category равно "нет детей"
data[data['children_category'] == 'нет детей']['children'].unique()

array([0], dtype=uint8)

Третий вопрос кейса звучит так: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

Для анализа необходимо категоризировать значения столбца `total_income`. Для этого исследуем столбец на предмет ключевых значений диапозона

In [25]:
print('Максимальный доход', data['total_income'].max())
print('Медианный доход', data['total_income'].median())
print('Минимальный доход', data['total_income'].min())

Максимальный доход 2265604
Медианный доход 142594.0
Минимальный доход 20667


Доход колеблется между 20 тысячами и 2.2 миллионами. Предлагается использовать следующие категории:
- До 80 тысяч
- От 80 до 150 тысяч
- От 150 до 250 тысяч
- От 150 до 500 тысяч
- От 500 тысяч до 1 миллиона
- Более 1 миллиона

In [26]:
#пишем функцию для категоризации значений столбца total_income
def income_to_category(row):
    income = row['total_income']
    
    if income < 80000:
        return 'до 80 т.'
    if income < 150000:
        return 'от 80 т. до 150 т.'
    if income < 250000:
        return 'от 150 т. до 250 т.'
    if income < 500000:
        return 'от 250 т. до 500 т.'
    if income < 1000000:
        return 'от 500 т. до 1 млн.'
    return 'более 1 млн.'

In [27]:
#применяем функцию для создания столбца income_category
data['income_category'] = data.apply(income_to_category, axis = 1)

#проверка
data

,children,family_status,income_type,debt,total_income,purpose,autofill_id,purpose_category,children_category,income_category
0,1,женат / замужем,сотрудник,0,253875,покупка жилья,NaN,жилая недвижимость,есть дети,от 250 т. до 500 т.
1,1,женат / замужем,сотрудник,0,112080,приобретение автомобиля,NaN,автомобиль,есть дети,от 80 т. до 150 т.
2,0,женат / замужем,сотрудник,0,145885,покупка жилья,NaN,жилая недвижимость,нет детей,от 80 т. до 150 т.
3,3,женат / замужем,сотрудник,0,267628,дополнительное образование,NaN,образование,есть дети,от 250 т. до 500 т.
4,0,гражданский брак,пенсионер,0,158616,сыграть свадьбу,NaN,свадьба,нет детей,от 150 т. до 250 т.
...,...,...,...,...,...,...,...,...,...,...
21516,1,гражданский брак,компаньон,0,224791,операции с жильем,NaN,жилая недвижимость,есть дети,от 150 т. до 250 т.
21517,0,женат / замужем,пенсионер,0,155999,сделка с автомобилем,NaN,автомобиль,нет детей,от 150 т. до 250 т.
21518,1,гражданский брак,сотрудник,1,89672,недвижимость,NaN,прочая недвижимость,есть дети,от 80 т. до 150 т.
21519,3,женат / замужем,сотрудник,1,244093,на покупку своего автомобиля,NaN,автомобиль,есть дети,от 150 т. до 250 т.


In [28]:
#проверка правильности дипозонов дохода и категорий
data.groupby('income_category').agg({'total_income' : ['min', 'max']})

total_income         
                             min      max
income_category                          
более 1 млн.             1004476  2265604
до 80 т.                   20667    79973
от 150 т. до 250 т.       150001   249991
от 250 т. до 500 т.       250130   499924
от 500 т. до 1 млн.       502318   997014
от 80 т. до 150 т.         80039   149993

Для наглядности отображения данных в виде сводной таблицы нам также потребуется ввести словестные категории столбца `debt` который сейчас имеет значение 0 и 1, где, очевидно, 0 соответствует отсутствию просрочек, а 1 - наличию. Используем следующие значения: 

- есть долги
- нет долгов

In [29]:
#пишем функцию для категоризации столбца debt
def debt_to_category(row):
    debt = row['debt']
    if debt == 1:
        return 'есть долги'
    return 'нет долгов'

#применяем функцию
data['debt_category'] = data.apply(debt_to_category, axis = 1)

#проверка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21521 entries, 0 to 21520
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   children           21521 non-null  uint8   
 1   family_status      21521 non-null  category
 2   income_type        21521 non-null  category
 3   debt               21521 non-null  uint8   
 4   total_income       21521 non-null  uint32  
 5   purpose            21521 non-null  object  
 6   autofill_id        2173 non-null   float64 
 7   purpose_category   21521 non-null  object  
 8   children_category  21521 non-null  object  
 9   income_category    21521 non-null  object  
 10  debt_category      21521 non-null  object  
dtypes: category(2), float64(1), object(5), uint32(1), uint8(2)
memory usage: 1.2+ MB


**Вывод**

Для упрощения анализа были добавлены следующие столбцы и введены следующие категории:

1. Столбец `children_category` для категоризации столбца `children` со следующими значениями:
- есть дети
- нет детей
2. Столбец `income_category` для категоризации столбца `total_income` со следующими значениями:
- До 80 тысяч
- От 80 до 150 тысяч
- От 150 до 250 тысяч
- От 150 до 500 тысяч
- От 500 тысяч до 1 миллиона
- Более 1 миллиона
3. Столбец `debt_category` для категоризации столбца `debt` со следующими значениями:
- есть долги
- нет долгов

<a id="main"></a>
## Ответы на вопросы кейса

Для ответа на вопросы кейса нагляднее всего использовать сводные таблицы. Для упрощения этого процесса создадим функцию, которая будет создавать сводную таблицу для конкретного категориального столбца.

In [30]:
#создаем функцию с двумя вводными переменными - df для диапозона данных и category для названия столбца
def to_pivot(df, category):
    pivot = df.pivot_table(index = category, columns = 'debt_category', values = 'debt', aggfunc = 'count')
    pivot['итого'] = pivot['нет долгов'] + pivot['есть долги']
    pivot['% должников'] = pivot['есть долги'] / pivot['итого'] * 100
    return pivot

Также для ответов на вопросы кейса необходимо понять, каково среднее значение доли должников по всему диапозону

In [31]:
debt_ratio = data['debt'].sum() / data['debt'].count()
print('Среднее количество должников по всей выборке: {:.0%}'.format(debt_ratio))

Среднее количество должников по всей выборке: 8%


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
pivot_children = to_pivot(data, 'children_category')
pivot_children

debt_category,есть долги,нет долгов,итого,% должников
children_category,,,,
есть дети,678,6698,7376,9.191974
нет детей,1063,13082,14145,7.515023


**Вывод**

Значение доли должников от всей выборки заемщиков с детьми на 1.2% выше среднего значения, а значение доли заемщиков без детей на 0.5% ниже.

Можно однозначно утверждать, что заемщики с детьми чаще не отдают кредиты, чем заемщики без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
pivot_family = to_pivot(data, 'family_status')
pivot_family

debt_category,есть долги,нет долгов,итого,% должников
family_status,,,,
Не женат / не замужем,274,2538,2812,9.743954
в разводе,85,1110,1195,7.112971
вдовец / вдова,63,897,960,6.562500
гражданский брак,388,3789,4177,9.288963
женат / замужем,931,11446,12377,7.522017


**Вывод**

Чаще всего допускают просрочку по кредиту люди, которые указывают свой статус как "не женат / не замужем" и "гражданский брак" - на 1.7% и 1.3% соответственно. Реже всего допускают просрочки вдовцы на 1.5%, и чуть реже среднего попадают в должники люди, указывающие статус "в разводе" и "женат / за мужем" на 0.9% и 0.5% соответственно.

Заивисмости однозначно присутствует и она не очевидна.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
pivot_income = to_pivot(data, 'income_category')
pivot_income

debt_category,есть долги,нет долгов,итого,% должников
income_category,,,,
более 1 млн.,2,23,25,8.000000
до 80 т.,174,2101,2275,7.648352
от 150 т. до 250 т.,569,6456,7025,8.099644
от 250 т. до 500 т.,180,2410,2590,6.949807
от 500 т. до 1 млн.,12,185,197,6.091371
от 80 т. до 150 т.,804,8605,9409,8.545010


**Вывод**

Реже всего на целых 2% допускают просрочки люди с доходами от 500 тысяч до 1 миллиона. Реже среднего попадают в должники люди с доходами до 80 тысяч и в диапозоне до 250 тысяч до 500 тысяч на 0.4% и 1.1% соотвественно. 

Люди с доходами от 80 тысяч до 150 тысяч чаще всего допускают просрочки на 0.5%, а люди в категории от 150 тысяч до 250 тысяч и более 1 миллиона рублей находятся в среднем значении.

Зависимости скорее отсутствует, колебания показателей незначительны. 

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
pivot_purpose = to_pivot(data, 'purpose_category')
pivot_purpose

debt_category,есть долги,нет долгов,итого,% должников
purpose_category,,,,
автомобиль,403,3911,4314,9.341678
жилая недвижимость,273,3588,3861,7.070707
образование,370,3651,4021,9.201691
прочая недвижимость,474,5892,6366,7.445806
ремонт,35,576,611,5.728314
свадьба,186,2162,2348,7.921635


**Вывод**

Чаще всего допускают просрочку люди, которые берут деньги на операции с личными автомобилем и на образование на 1.3% и 1.2% соответственно. Реже всего в должники попадают люди, которые совершают различные операции с недвижимостью. Рекордно низкое значение отмечается на показателе "ремонт" - 5.7% против 8% среднего. 

Люди, которые берут деньги на свадьбу находятся в рамках среднего значения.

<a id="final"></a>
## Общий вывод

Исследуемый диапозон данных имел явные пропуски (около 10% от всего объема информации) в столбце `total_income` - ежемесячный доход. Пропуски были заполнены медианными значениями, найденными по категориям столбца `income_type` - типа занятости. Впроцессе предоработки данных удалено всего 4 строки, три из которых оказались дубликатами.

Для удобства проведения аналитической работы данные были категоризированы следующим образом: 

__Для столбца `purpose` - цель получения кредита был создан столбец `purpose_category`:__
- Жилая недвижимость для всех предложений, где есть слово жилье (кроме ремонта)
- Прочая недвижимость для всех предложений, где есть слово недвижмость (кроме жилья и ремонта)
- Ремонт
- Свадьба
- Образование
- Автомобиль

__Для столбца `total_income` - ежемесячный доход был создан столбец `income_category`:__
- До 80 тысяч
- От 80 до 150 тысяч
- От 150 до 250 тысяч
- От 150 до 500 тысяч
- От 500 тысяч до 1 миллиона
- Более 1 миллиона

__Для столбца `children` - количество детей в семье был создан столбец `children_category`:__
- Есть дети
- Нет детей

__Для столбца `debt` - наличие задолженности был создан столбец `debt_category`:__
- Есть долги
- Нет долгов

Перед анализом запрашиваемых выборок был оценен средний по всему объему данных процент должников, он составляет **8%**

Анализ осуществлен с помощью составления сводных таблиц, результат можно посмотреть в [части 3](#main)

**В результате анализа данных можно сказать следующее:**

Замещики с детьми, не женатые или состоящие в гражданском браке, а также берущие заемы на операции с личными автомобилем и на образование чаще не отдают кредиты в положенный срок, чем остальные примерно на 12-20%. 

Люди, находящиеся в разводе, вдовцы и те, кто берет займы для осуществления ремонта жилья чаще остальных возвращают кредиты в срок, на этот вывод может повлиять относительно небольшая выборка, требуется исследование на большей выборке.

Показатель уровня дохода скорее не влияет или влияет незначительно на вероятность возврата кредита в срок.